# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846800067508                   -0.70    4.5         
  2   -7.852296431475       -2.26       -1.53    1.0   33.0ms
  3   -7.852614098608       -3.50       -2.56    1.8   35.3ms
  4   -7.852645789342       -4.50       -2.87    2.5   40.9ms
  5   -7.852646462763       -6.17       -3.16    1.0   31.6ms
  6   -7.852646677810       -6.67       -4.30    1.2   32.4ms
  7   -7.852646686661       -8.05       -5.05    2.5   42.3ms
  8   -7.852646686723      -10.21       -5.34    1.5   37.9ms
  9   -7.852646686729      -11.24       -5.84    1.0   30.3ms
 10   -7.852646686730      -12.13       -6.23    1.5   32.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846835133184                   -0.70           5.0         
  2   -7.852526363271       -2.24       -1.63   0.80    2.0    321ms
  3   -7.852636815897       -3.96       -2.71   0.80    1.0   27.5ms
  4   -7.852646472734       -5.02       -3.28   0.80    2.2   37.2ms
  5   -7.852646667772       -6.71       -4.13   0.80    1.0   27.9ms
  6   -7.852646686384       -7.73       -4.68   0.80    1.8   33.5ms
  7   -7.852646686704       -9.49       -5.41   0.80    1.2   30.2ms
  8   -7.852646686729      -10.60       -6.58   0.80    1.8   34.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.366733e+01     3.314700e+00
 * time: 0.4878859519958496
     1     1.263723e+00     1.804067e+00
 * time: 0.7247071266174316
     2    -1.827860e+00     2.220975e+00
 * time: 0.7536890506744385
     3    -3.598623e+00     1.941920e+00
 * time: 0.7949259281158447
     4    -4.970364e+00     1.771720e+00
 * time: 0.8364410400390625
     5    -6.557245e+00     1.353857e+00
 * time: 0.8776960372924805
     6    -7.349210e+00     7.287997e-01
 * time: 0.9183769226074219
     7    -7.618835e+00     6.589671e-01
 * time: 0.9472310543060303
     8    -7.722296e+00     3.189256e-01
 * time: 0.9763400554656982
     9    -7.775083e+00     1.256551e-01
 * time: 1.0054590702056885
    10    -7.809851e+00     9.843377e-02
 * time: 1.0340509414672852
    11    -7.829770e+00     6.890962e-02
 * time: 1.0627870559692383
    12    -7.842515e+00     4.768674e-02
 * time: 1.0913739204406738
    13    -7.850250e+00     2.751567e-02
 * time: 1.1203501224

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846813738911                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645913119                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.94s
  3   -7.852646686730      -13.28       -7.25    255ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.3193862884767917e-7
|ρ_newton - ρ_scfv| = 1.9413131601378368e-7
|ρ_newton - ρ_dm|   = 9.308931891717643e-10
